In [63]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
import pandas as pd
from functions import main_cleaning

pd.set_option('display.max_columns', None) 

df_raw = pd.read_csv("dielectron.csv")

In [65]:
df_cleaned = main_cleaning(df_raw)
df_cleaned.reset_index(drop=True, inplace=True)

e:\Iván\Documentos\Clase\Informatica\Beca IronHack\Temario\PROYECTOS\Proyecto_final_1710\functions.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["is_same_charge"] = df["Q1"] == df["Q2"]
e:\Iván\Documentos\Clase\Informatica\Beca IronHack\Temario\PROYECTOS\Proyecto_final_1710\functions.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["is_outlier"] = False


# MACHINE LEARNING

### Los resultados de MSE, MAE y R2 para KNN sin normalizar eran: 
MAE: 17.717622715597013  
MSE: 12.581512831873468   
R2: 0.507506217561434  
### So I will normalize/standarize.

In [66]:
df_ml = df_cleaned.dropna()

In [67]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score

In [68]:
features_lower = df_ml[["Run", "Q1", "pz1", "pz2", "Q2", "is_outlier"]]
target_lower = df_ml[["M"]]

KeyError: "['Q1pz1'] not in index"

In [51]:
from sklearn.model_selection import train_test_split

X_train_lower, X_test_lower, y_train_lower, y_test_lower = train_test_split(features_lower, target_lower, test_size=0.20, random_state=42)

In [52]:
scaler_lower = StandardScaler()

X_train_lower_scaled = scaler_lower.fit_transform(X_train_lower)        

X_test_lower_scaled = scaler_lower.transform(X_test_lower)

In [53]:
X_train_lower_scaled = pd.DataFrame(X_train_lower_scaled, columns = X_train_lower.columns)   
X_test_lower_scaled = pd.DataFrame(X_test_lower_scaled, columns = X_test_lower.columns)

In [54]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.preprocessing import StandardScaler
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.linear_model import LinearRegression
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.svm import SVR
# from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score
# import xgboost as xgb

# # Define models
# models = {
#   'KNeighborsRegressor': KNeighborsRegressor(),
#   'Linear Regression': LinearRegression(),
#   'Decision Tree': DecisionTreeRegressor(random_state=42),
#   'Random Forest': RandomForestRegressor(random_state=42, n_estimators=100),
#   'SVR': SVR(),
#   'XGBoost': xgb.XGBRegressor()
# }   

# # Train and evaluate models
# results = {}

# for name, model in models.items():
#   # Train the model
#     model.fit(X_train_lower_scaled, y_train_lower)
  
#   # Make predictions
#     y_pred_lower_scaled = model.predict(X_test_lower_scaled)

#     mae_standardized = mean_absolute_error(y_test_lower, y_pred_lower_scaled)
#     mse_standardized = root_mean_squared_error(y_test_lower, y_pred_lower_scaled)
#     r2_standardized = r2_score(y_test_lower, y_pred_lower_scaled)
  
#   # Store results
#     results[name] = {
#         'mae' : mae_standardized,
#         'mse' : mse_standardized,
#         'R2' : r2_standardized
#   }

# # Print results
# for name, result in results.items():
#   print(f"\n{name}:")
#   print(f"Mean Absolute Error: {result['mae']}")
#   print(f"Root Mean Squared Error: {result['mse']}")
#   print(f"R2: {result['R2']}")

# # Compare R2_value
# r2_values = {name: result['R2'] for name, result in results.items()}
# best_model = max(r2_values, key=r2_values.get)

# print("\nModel Accuracy Comparison:")
# for name, accuracy in r2_values.items():
#   print(f"{name}: {accuracy:.4f}")

# print(f"\nBest performing model: {best_model} with accuracy {r2_values[best_model]:.4f}")

### CON TODO 
Random Forest:
Mean Absolute Error: 2.3478761082736876  
Root Mean Squared Error: 3.918134937140667  
R2: 0.9759148902055805  

XGBoost:
Mean Absolute Error: 2.6338879782682474  
Root Mean Squared Error: 3.783981831730138  
R2: 0.9775359561492042  

### SOLO PZ

In [55]:
# from sklearn.model_selection import RandomizedSearchCV
# import numpy as np

# rf_lower = RandomForestRegressor()

# grid = {
#     "n_estimators": [int(x) for x in np.linspace(start=10, stop=210, num=10)],
#     "max_depth": [None, 5],
#     "max_leaf_nodes": [None, 10, 50, 100],
#     "min_samples_split": [2, 5, 10],
#     "min_samples_leaf": [1, 2, 5]
# }


# model = RandomizedSearchCV(estimator = rf_lower, param_distributions = grid, n_iter = 10, cv = 5, n_jobs = -1) 


Solo pz con ese grid -> (9.478894753076418, 15.181723878082533, 0.6383969959427274)  

Best params:  
{'n_estimators': 143,
 'min_samples_split': 10,
 'min_samples_leaf': 5,
 'max_leaf_nodes': 100,
 'max_depth': None}

In [56]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
import xgboost as xgb

xg_lower = xgb.XGBRegressor()

grid = {
    'n_estimators': [100, 200, 300, 400, 500],            # Número de árboles
    'max_depth': [3, 5, 7, 9, 11],                        # Profundidad máxima de los árboles
    'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],        # Tasa de aprendizaje
    'subsample': [0.6, 0.8, 1.0],                         # Proporción de muestras utilizadas para entrenar
    'colsample_bytree': [0.6, 0.8, 1.0],                 # Proporción de características utilizadas para entrenar
    'gamma': [0, 0.1, 0.2, 0.3],                          # Reducción de pérdida requerida para hacer una partición adicional
    'reg_alpha': [0, 0.1, 1, 10],                         # Regularización L1
    'reg_lambda': [0, 0.1, 1, 10]                        # Regularización L2
}


model = RandomizedSearchCV(estimator = xg_lower, param_distributions = grid, n_iter = 10, cv = 5, n_jobs = -1) 

In [57]:
# model.best_params_

In [58]:
model.fit(X_train_lower_scaled, y_train_lower)
  
y_pred_lower_scaled = model.predict(X_test_lower_scaled)

mae_standardized = mean_absolute_error(y_test_lower, y_pred_lower_scaled)
mse_standardized = root_mean_squared_error(y_test_lower, y_pred_lower_scaled)
r2_standardized = r2_score(y_test_lower, y_pred_lower_scaled)

mae_standardized, mse_standardized, r2_standardized

(9.169353184015405, 14.906887308065759, 0.6513707755273004)

In [59]:
# model.best_params_

In [60]:
features_scaled = scaler_lower.transform(features_lower)

y_pred_lower_real = model.predict(features_scaled)

mae_standardized = mean_absolute_error(target_lower, y_pred_lower_real)
mse_standardized = root_mean_squared_error(target_lower, y_pred_lower_real)
r2_standardized = r2_score(target_lower, y_pred_lower_real)

mae_standardized, mse_standardized, r2_standardized

(9.05175301056049, 14.708094083162141, 0.6608511588229353)

In [61]:
from sklearn.model_selection import cross_val_score

# Aplicamos la validación cruzada en el conjunto de entrenamiento
scores = cross_val_score(model, X_train_lower_scaled, y_train_lower, cv=5, scoring='r2')

print("Cross-validated R2 scores:", scores)
print("Mean cross-validated R2:", scores.mean())


Cross-validated R2 scores: [0.66011916 0.65262453 0.65122104 0.6492082  0.63502745]
Mean cross-validated R2: 0.6496400767316656


Sin px -> Mean cross-validated R2: 0.8715048380715927

Sin px ni py -> Mean cross-validated R2: 0.6095551354410731

Sin px ni py (XGBoost) -> Mean cross-validated R2: 0.6483473040652855
